<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/bert-paradetox-1Token-split-masked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install transformers[torch]

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/paradetox-1Token-Split")

In [10]:
'''
# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Splitting the dataset into train, validation, and test
train_testvalid = dataset["train"].train_test_split(test_size=0.3)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)

# Creating a DatasetDict
split_dataset = DatasetDict({
    "train": train_testvalid["train"],
    "validation": test_valid["test"],
    "test": test_valid["train"]
})

split_dataset

split_dataset.push_to_hub("HamdanXI/paradetox-1Token-Split")
'''

In [5]:
# Function to replace tokens
def replace_tokens(example):
    example['masked_comment'] = example['masked_comment'].replace('[REPLACE]', '[MASK]').replace('[DELETE]', '[MASK]')
    return example

# Apply the function to each example in the dataset
dataset = dataset.map(replace_tokens)

In [6]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize and create labels
def tokenize_and_create_labels(examples):
    # Tokenize the masked comments
    tokenized_input = tokenizer(examples["masked_comment"], padding="max_length", truncation=True, return_tensors="pt")
    # Tokenize the original comments (choose the appropriate field)
    tokenized_labels = tokenizer(examples["en_neutral_comment"], padding="max_length", truncation=True, return_tensors="pt")

    labels = tokenized_input.input_ids.detach().clone()
    # Create labels
    mask = tokenized_input.input_ids != tokenized_labels.input_ids
    labels[~mask] = -100  # Set labels to -100 where the tokens are not masked

    tokenized_input["labels"] = labels
    return tokenized_input

# Tokenize the dataset and create labels
tokenized_datasets = dataset.map(tokenize_and_create_labels, batched=True)

# Load pre-trained BERT model for masked language modeling
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define training arguments
training_args = TrainingArguments(
    output_dir="bert-paradetox-1Token-split-masked",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    push_to_hub=True,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Train the model
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss
1,No log,0.001366
2,No log,0.000606
3,0.334900,0.000491


TrainOutput(global_step=711, training_loss=0.2357878524668609, metrics={'train_runtime': 435.1736, 'train_samples_per_second': 26.086, 'train_steps_per_second': 1.634, 'total_flos': 2987901034905600.0, 'train_loss': 0.2357878524668609, 'epoch': 3.0})